In [14]:
import dash
from dash import dcc, html, Input, Output
import plotly.express as px
import pandas as pd
import os
from flask import send_from_directory

df = pd.read_csv('pokemon.csv')
df['image_path'] = 'archive (1)/images/' + df['name'] + '/0.jpg'

app = dash.Dash(__name__)
server = app.server


@server.route('/images/<pokemon>/<image>')
def serve_image(pokemon, image):
    return send_from_directory(os.path.join('archive (1)/images', pokemon), image)


fig = px.scatter(
    df,
    x='attack',
    y='defense',
    hover_name='name',
    color='type1',
    title='Pokemon Attack vs Defense'
)

app.layout = html.Div([
    html.H1('Pokemon Visual Pokedex'),
    dcc.Graph(id='scatter-plot', figure=fig),
    html.Div(id='pokemon-details', style={'marginTop': 20, 'color': 'white'})
])


@app.callback(
    Output('pokemon-details', 'children'),
    Input('scatter-plot', 'clickData')
)
def display_pokemon_details(clickData):
    if clickData is None:
        return 'Click a Pokémon on the graph to see details.'

    name = clickData['points'][0]['hovertext']
    pokemon = df[df['name'] == name].iloc[0]

    image_url = f'/images/{name}/0.jpg'

    details = [
        html.H3(name),
        html.Img(src=image_url, style={'height': '200px'}),
        html.Ul([
            html.Li(f'HP: {pokemon["hp"]}'),
            html.Li(f'Attack: {pokemon["attack"]}'),
            html.Li(f'Defense: {pokemon["defense"]}'),
            html.Li(f'Sp. Attack: {pokemon["sp_attack"]}'),
            html.Li(f'Sp. Defense: {pokemon["sp_defense"]}'),
            html.Li(f'Speed: {pokemon["speed"]}'),
            html.Li(f'Primary Type: {pokemon["type1"]}'),
            html.Li(f'Secondary Type: {pokemon["type2"] if pd.notnull(pokemon["type2"]) else "None"}'),
        ])
    ]

    return html.Div(details)


if __name__ == '__main__':
    app.run(debug=True)
